In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
from pathlib import Path
from PIL import Image
import os
!pip install split-folders
import splitfolders
import json
import shutil


In [ ]:
Select_class = [1, 6, 7, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 26, 27, 31, 36, 38, 43, 45, 46, 55, 58, 60, 61, 64, 68, 80, 81, 86, 87, 88, 91, 97, 98, 101,
                113, 130, 150, 154, 155, 162, 168, 169, 173, 187, 190, 201]

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/NutritionExperts/config.json') as file:
    config = json.load(file)

config_dataset = config['dataset']

DATASETS_PATH = config_dataset['datasets_path']
FOOD256_PATH = config_dataset['UECFOOD256_path']


In [ ]:
def copy_images(src_directory, dest_directory):
    src_directory = Path(src_directory)
    dest_directory = Path(dest_directory)

    # Create the target folder 'image'
    dest_subdir = dest_directory / 'images'
    dest_subdir.mkdir(parents=True, exist_ok=True)
    image_formats = ['.jpg', '.png', '.jpeg', '.gif']

    # Extract the original class name (last folder name)
    original_class_name = src_directory.name.split('/')[-1]

    # Copy all matching format image files and rename them
    for img_file in src_directory.iterdir():
        if img_file.is_file() and img_file.suffix.lower() in image_formats:
            # Extract the filename without the extension
            original_filename = img_file.stem

            # Create the new filename with the desired prefix and suffix
            new_filename = f"{original_filename}_{original_class_name}.jpg"

            # Get the path for the renamed file within the destination directory
            renamed_dest_path = dest_subdir / new_filename

            # Copy the image file to the destination directory with the new name
            shutil.copy2(img_file, renamed_dest_path)

In [ ]:
def copy_txt_files(src_directory, dest_directory, counter):
    src_directory = Path(src_directory)
    dest_directory = Path(dest_directory)
    for txt_file in src_directory.glob('*.txt'):
        with open(txt_file, 'r') as f:
            lines = f.readlines()
        lines = lines[1:]
        for line in lines:
            # first set of numbers to be used as the filename
            original_first_number = line.split()[0]

            #  the name of the source folder
            line_parts = line.split()
            line_parts[0] = str(counter)
            new_line = " ".join(line_parts)
            # Extract the last word from the src_directory to be included in the new filename
            original_class_name = src_directory.name.split('/')[-1]

            new_file_name = f"{original_first_number}_{original_class_name}.txt"
            dest_file = dest_directory / new_file_name

            with open(dest_file, 'w') as new_file:
                new_file.write(new_line)

In [ ]:
def convert_label(name, x1, y1, x2, y2, image_width, image_height):
    '''
    Normalized(Xmin) = (Xmin+w/2)/Image_Width

    Normalized(Ymin) = (Ymin+h/2)/Image_Height

    Normalized(w) = w/Image_Width

    Normalized(h) = h/Image_Height
    '''
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    w = x2 - x1
    h = y2 - y1
    normalizedXmin = (x1 + w/2) / image_width
    normalizedYmin = (y1 + h/2) / image_height
    normalizedW = w / image_width
    normalizedH = h / image_height
    return (name, str(normalizedXmin), str(normalizedYmin), str(normalizedW), str(normalizedH))


In [ ]:
def write_to_txt(file, converted_labels):
    labels_directory = FOOD256_PATH + '/labels'
    fileName = str(file).split('/')[-1]
    f = open(labels_directory + '/' + fileName, "w")
    for converted_label in converted_labels:
        line = ' '.join(converted_label)
        f.write(line + '\n')
    f.close()

In [ ]:
def generate_yolo_labels():
    labels_directory = FOOD256_PATH + '/labels'
    if not os.path.exists(labels_directory):
        os.makedirs(labels_directory)
    annotations_directory = FOOD256_PATH + '/Annotations'
    images_directory = FOOD256_PATH + '/images'
    # iterate over files in that directory
    files = Path(annotations_directory).glob('*')
    for file in files:
        # Load the corresponding image to get the image size
        image_file = Path(images_directory) / (file.stem + ".jpg")
        image = Image.open(image_file)
        image_width, image_height = image.size

        lines = file.read_text().split('\n')
        converted_labels = []

        for line in lines:
            if line == '':
                break
            line = line.strip()
            name = line.split(' ')[0]
            x1 = line.split(' ')[1]
            y1 = line.split(' ')[2]
            x2 = line.split(' ')[3]
            y2 = line.split(' ')[4]
            converted_label = convert_label(name, x1, y1, x2, y2, image_width, image_height)
            converted_labels.append(converted_label)
        write_to_txt(file, converted_labels)

In [ ]:
def train_val_test_split(input_folder, output):
    # Split with a ratio.
    splitfolders.ratio(input_folder, output=output,
        seed=1337, ratio=(.9, .1), group_prefix=None, move=False)



In [ ]:
def copy_folder_contents(source_folders, destination_folders):
    for i in range(len(source_folders)):
        source_folder = source_folders[i]
        destination_folder = destination_folders[i]
        if os.path.exists(destination_folder):
            shutil.rmtree(destination_folder)
        os.makedirs(destination_folder)

        # Copy the contents of the source folder to the target folder
        for root, dirs, files in os.walk(source_folder):
            for file in files:
                source_file = os.path.join(root, file)
                destination_file = os.path.join(destination_folder, file)
                shutil.copy(source_file, destination_file)


In [ ]:
def delete_folders(folders):
    for folder in folders:
        shutil.rmtree(folder)

In [ ]:
import torch
USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
print('using device:', device)

using device: cuda


In [ ]:
if __name__ == '__main__':
    # rename school_lunch/Images to school_lunch/images for yolo dataset formatting
    try:
        os.rename(FOOD256_PATH + '/Image', FOOD256_PATH + '/image')
    except:
        print('Capitalized Images folder not found or already converted to lower case!')
    counter = 0
    for i in Select_class:
        copy_txt_files('/content/drive/MyDrive/Colab Notebooks/datasets/UECFOOD256/' + str(i), FOOD256_PATH + '/Annotations', counter)
        copy_images('/content/drive/MyDrive/Colab Notebooks/datasets/UECFOOD256/' + str(i), FOOD256_PATH)
        counter = counter + 1
    generate_yolo_labels()
    train_val_test_split(FOOD256_PATH, DATASETS_PATH + '/yolo_UECFOOD256')

  #   # organize the folders
  # source_folders = ['/content/drive/MyDrive/Colab Notebooks/datasets/data/train/image', '/content/drive/MyDrive/Colab Notebooks/datasets/data/test/image', '/content/drive/MyDrive/Colab Notebook/datasets/data/val/image', '/content/drive/MyDrive/Colab Notebooks/datasets/data/train/label', '/content/drive/MyDrive/Colab Notebooks/datasets/data/test/label', '/content/drive/MyDrive/Colab Notebooks/datasets/data/val/label']
  # destination_folders = ['/content/drive/MyDrive/Colab Notebooks/datasets/data/images/train', '/content/drive/MyDrive/Colab Notebooks/datasets/data/images/test', '/content/drive/MyDrive/Colab Notebooks/datasets/data/images/val', '/content/drive/MyDrive/Colab Notebooks/datasets/data/labels/train', '/content/drive/MyDrive/Colab Notebooks/datasets/data/labels/test', '/content/drive/MyDrive/Colab Notebooks/datasets/data/labels/val']
  # copy_folder_contents(source_folders, destination_folders)
  # folders_to_delete = ['/content/drive/MyDrive/Colab Notebooks/datasets/data/train', '/content/drive/MyDrive/Colab Notebooks/datasets/data/test', '/content/drive/MyDrive/Colab Notebooks/datasets/data/val']
  # delete_folders(folders_to_delete)

Capitalized Images folder not found or already converted to lower case!


Copying files: 60796 files [16:19, 62.04 files/s]


In [ ]:
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
device

device(type='cuda')